## Computer vision application
## Data modeling

This project aims to explore the computer vision field, from image processing operations to image classification. The data used here is made upon *2527 different images of garbage from 6 different classes*: cardboard, glass, metal, paper, plastic, trash. The ultimate goal is to develop an image classifier model that, after trained and validated using specific datasets, is able to predict the probabilities that a given image belongs to each of those classes. Again, a specific test dataset is applied to evaluate model performance. To achieve that goal, different code artifacts were generated:
* Python modules for image processing and to support model development.
* Notebooks of data engineering and data preparation that apply different operations over images (flip, crop, change in brightness, and so on).
* Notebooks of data modeling that implement experimentation to find the best image classifier model, analysis of the most relevant models and creation of codes to apply the best one in production.

-----------

This notebook imports training, validation and test images, then applies some data preparation procedures (image scaling, data augmentation). The core of this notebook consists of codes to implement the construction of image classifiers based on different learning methods: machine learning (logistic regression and LightGBM), Artificial Neural Networks (ANN), Convolutional Neural Networks (CNN) and Transfer Learning. This notebook also exports model artifacts and the assessment of its performance, besides of presenting the structure and test accuracy of all constructed models.

**Sumário:**
1. [Libraries](#libraries)<a href='#libraries'></a>.
2. [Functions and classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Settings](#settings)<a href='#settings'></a>.
4. [Importing the data](#data_imports)<a href='#data_imports'></a>.
  * [Data importing and cleaning](#data_imp_clean)<a href='#data_imp_clean'></a>.


5. [Preparing the data](#data_prep)<a href='#data_prep'></a>.
  * [Data augmentation](#data_aug)<a href='#data_aug'></a>.
  * [Data scaling](#scaling)<a href='#scaling'></a>.
  * [Flattening images](#flattening_imgs)<a href='#flattening_imgs'></a>.


6. [Machine learning](#ml)<a href='#ml'></a>.
7. [Artificial neural networks](#ann)<a href='#ann'></a>.
8. [Convolutional neural networks](#cnn)<a href='#cnn'></a>.
9. [Transfer learning](#transfer_learning)<a href='#transfer_learning'></a>.
10. [Exporting outcomes](#exports)<a href='#exports'></a>.
11. [Model assessment](#model_assess)<a href='#model_assess'></a>.

<a id='libraries'></a>

## Libraries





In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd "/content/gdrive/MyDrive/Studies/computer_vision/notebooks/"

/content/gdrive/MyDrive/Studies/computer_vision/notebooks


In [ ]:
# !pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import time
import cv2
from copy import deepcopy
import pickle

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform, randint

import lightgbm as lgb

from keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16, VGG19, Xception

In [ ]:
import sys

sys.path.append(
    os.path.abspath(
        os.path.join(
            os.path.dirname(__doc__), '../src'
        )
    )
)

<a id='functions_classes'></a>

## Functions and classes

In [ ]:
from utils import frequency_list, create_grid, running_time
from data_eng import ImportCleanImages
from data_augmentation import KerasImageAugment, Brightness, Flip, Rotation, Shift, Crop, Blur, ImageAugment
from transformations import ScaleImage
from neural_networks import AccuracyEpochs

<a id='settings'></a>

## Settings

<a id='files_manag'></a>

### Files management

In [ ]:
# Declare whether outcomes should be exported:
export = True

# Declare if images should be imported in grey scale:
grey_scale = False

# Identification of experiment:
experiment_id = str(int(time.time()))

# Declare whether model artifacts should be exported:
export_artifacts = True

<a id='data_prep_settings'></a>

### Data preparation settings


In [ ]:
# Declare whether to perform Keras-based data augmentation:
keras_augment = True

# Declare whether to perform custom data augmentation:
custom_augment = False

# Declare whether to scale images:
scale = True

#### Data augmentation settings

In [ ]:
# Parameters for Keras data augmentation:
keras_oper_params = {
    'augmentation_factor': 1.5,
    'oper_params': {
        'brightness_range': [0.5, 1.5],
        'horizontal_flip': True,
        'vertical_flip': True,
        'rotation_range': 180,
        # 'width_shift_range': [-200,200],
        # 'height_shift_range': [-200,200],
        'zoom_range': [0.5, 1.75],
        'fill_mode': 'nearest'
    }
}

cust_oper_params = {
    'augmentation_factor': 1.5,
    'flip': {
        'horizontal': True,
        'vertical': True,
        'flip_prob': 0.5
    },
    'rotation': {
        'rotation_range': (0, 180),
        'rotation_prob': 0.25
    },
    'blur': {
        'kernel_size': (9, 9),
        'blur_prob': 0.1
    },
    'shift': {
        'tx_range': (-150, 150),
        'ty_range': (-150, 150),
        'shift_prob': 0.2
    },
    'brightness': {
        'change_range': (-0.5, 0.5),
        'bright_prob': 0.5
    },
    'crop': {
        'center_range': (0.5, 0.5),
        'window_range': (25, 150),
        'crop_prob': 0.25
    }
}

# Declare parameters of data augmentation:
AUGMENT_PARAMS = keras_oper_params if keras_augment else (cust_oper_params if custom_augment else {})

<a id='model_settings'></a>

### Model settings

In [ ]:
# Declare which model should be used for training:
which_model = 'transfer' # ['lr', 'lgb', 'ann', 'cnn', 'transfer']

#### Parameters optimization

In [ ]:
if which_model in ['lr', 'lgb']:
    # Grids of values of parameters under optimization for logistic regression:
    LR_PARAMS = {
        "C": [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.25, 0.3, 0.5, 0.75, 1, 3, 10],
        'penalty': ['l1'],
        'solver': ['liblinear'],
        'warm_start': [True]
    }

    # Grids of values of parameters under optimization for LightGBM:
    LGB_PARAMS = {
        'objective': ['multiclass'],
        'bagging_fraction': uniform(0.5, 0.5),
        # 'bagging_fraction': [0.7720050594069691],
        'learning_rate': uniform(0.0001, 0.1),
        # 'learning_rate': [0.06551373469707444],
        'max_depth': randint(1, 10),
        # 'max_depth': [6],
        'num_iterations': [100, 250, 500]
        # 'num_iterations': [100]
    }

<a id='image_files'></a>

### Image files

In [ ]:
# Available classes:
classes = os.listdir('../data/images/')

# Dictionary with label by class:
label_dict = dict(zip(classes, [i+1 for i in range(len(classes))]))
print(label_dict)

# File names of each dataset:
train_images = [f.split('.')[0] for f in list(pd.read_csv('../data/train_images.txt', sep=' ', header=None)[0])]
val_images = [f.split('.')[0] for f in list(pd.read_csv('../data/val_images.txt', sep=' ', header=None)[0])]
test_images = [f.split('.')[0] for f in list(pd.read_csv('../data/test_images.txt', sep=' ', header=None)[0])]

{'cardboard': 1, 'glass': 2, 'metal': 3, 'paper': 4, 'plastic': 5, 'trash': 6}


<a id='data_imports'></a>

## Importing the data

<a id='data_imp_clean'></a>

### Data importing and cleaning

In [ ]:
# Object for collecting and cleaning image data:
import_clean = ImportCleanImages(
    path_to_files='../data', images_folder='images', train_val_test_file='images',
    shuffle=True, label_dict=label_dict,
    grey_scale=grey_scale, resize=True, width=224, height=224
)
datasets = import_clean.build_datasets()

# Training, validation and test images, labels and ids:
images_train, labels_train, ids_train = datasets['train']
images_val, labels_val, ids_val = datasets['val']
images_test, labels_test, ids_test = datasets['test']

print(f'Shape of images_train: {images_train.shape}.')
print(f'Shape of images_val: {images_val.shape}.')
print(f'Shape of images_test: {images_test.shape}.')

Shape of images_train: (1768, 224, 224, 3).
Shape of images_val: (328, 224, 224, 3).
Shape of images_test: (431, 224, 224, 3).


#### Sanity checks

In [ ]:
if len(images_train)!=len(train_images):
  print('Inconsistent number of images between training data and predefined set of training images!')

if len(images_val)!=len(val_images):
  print('Inconsistent number of images between validation data and predefined set of validation images!')

if len(images_test)!=len(test_images):
  print('Inconsistent number of images between test data and predefined set of test images!')

if (len([c for c in ids_train if c not in train_images]) > 0) |\
  (len([c for c in train_images if c not in ids_train]) > 0):
  print('Some images were not found in training data or set of training images!')

if (len([c for c in ids_val if c not in val_images]) > 0) |\
  (len([c for c in val_images if c not in ids_val]) > 0):
  print('Some images were not found in validation data or set of validation images!')

if (len([c for c in ids_test if c not in test_images]) > 0) |\
  (len([c for c in test_images if c not in ids_test]) > 0):
  print('Some images were not found in test data or set of test images!')

<a id='data_prep'></a>

## Preparing the data

<a id='data_aug'></a>

### Data augmentation

#### Keras augmentation

In [ ]:
if keras_augment:
    # Object for implementing data augmentation:
    data_aug = KerasImageAugment(
        oper_params=keras_oper_params['oper_params'],
        augmentation_factor=keras_oper_params['augmentation_factor']
    )

    # Augmented batch of images:
    images_train_aug, labels_train = data_aug.augment_data(images_batch=images_train, labels=labels_train)

    print(f'Shape of images_train: {images_train.shape}.')
    print(f'Shape of images_train_aug: {images_train_aug.shape}.')
    print(f'Number of augmented labels: {len(labels_train)}.')

Shape of images_train: (1768, 224, 224, 3).
Shape of images_train_aug: (2652, 224, 224, 3).
Number of augmented labels: 2652.


#### Custom augmentation

In [ ]:
if custom_augment:
    # Object for implementing data augmentation:
    data_aug = ImageAugment(
        operations = (
            Flip(**cust_oper_params['flip']),
            Rotation(**cust_oper_params['rotation']),
            Blur(**cust_oper_params['blur']),
            Shift(**cust_oper_params['shift']),
            Brightness(**cust_oper_params['brightness']),
            Crop(**cust_oper_params['crop'])
        ),
        augmentation_factor=cust_oper_params['augmentation_factor']
    )

    # Augmented batch of images:
    images_train_aug, labels_train = data_aug.augment_data(images_batch=images_train, labels=labels_train)

    print(f'Shape of images_train: {images_train.shape}.')
    print(f'Shape of images_train_aug: {images_train_aug.shape}.')
    print(f'Number of augmented labels: {len(labels_train)}.')

<a id='scaling'></a>

### Data scaling

#### Normalization

In [ ]:
if scale:
    # Object for scaling data:
    scale_imgs = ScaleImage()

    # Scaled batch of images:
    images_train_scaled = scale_imgs.transform(
        images_batch=images_train_aug if (keras_augment | custom_augment) else images_train
    )
    images_val_scaled = scale_imgs.transform(images_batch=images_val)
    images_test_scaled = scale_imgs.transform(images_batch=images_test)

<a id='flattening_imgs'></a>

### Flattening images

In [ ]:
if which_model in ['lr', 'lgb', 'ann']:
    # Converting image data into vectors:
    flatten_images_train = pd.concat(
        [pd.DataFrame(np.ravel(img)).T for img in images_train_scaled],
        axis=0, sort=False
    ).reset_index(drop=True)
    print(f'Shape of flatten_images_train: {flatten_images_train.shape}.')

    flatten_images_val = pd.concat(
        [pd.DataFrame(np.ravel(img)).T for img in images_val_scaled],
        axis=0, sort=False
    ).reset_index(drop=True)
    print(f'Shape of flatten_images_val: {flatten_images_val.shape}.')

    flatten_images_test = pd.concat(
        [pd.DataFrame(np.ravel(img)).T for img in images_test_scaled],
        axis=0, sort=False
    ).reset_index(drop=True)
    print(f'Shape of flatten_images_test: {flatten_images_test.shape}.')

<a id='ml'></a>

## Machine learning

In [ ]:
### Incorporar às classes KfoldsCV e KfoldsCV_fit códigos para avaliação de modelos de classificação multinomial.

<a id='lr_model'></a>

### Logistic regression

#### Model configuration

In [ ]:
if which_model=='lr':
    # Creating grid of hyper-parameters for optimization:
    lr_grid = create_grid(params=LR_PARAMS, random_search=False)

#### Model training

In [ ]:
if which_model=='lr':
    lr_model = {}
    best_val_acc, model_params = 0, None

    start_time = datetime.now()

    # Loop over combinations of hyper-parameters:
    for params in lr_grid:
        # Creating and fitting the model:
        lr_model[str(params)] = LogisticRegression(**params)
        lr_model[str(params)].fit(X=flatten_images_train, y=labels_train)

        # Predictions on validation set:
        val_preds = lr_model[str(params)].predict(X=flatten_images_val)

        # Model performance on validation set:
        new_acc = accuracy_score(y_true=labels_val, y_pred=val_preds)
        if new_acc >= best_val_acc:
            model_params = deepcopy(params)
            best_val_acc = new_acc
    
    end_time = datetime.now()
    elapsed_time = running_time(start_time=start_time, end_time=end_time)

    print(f'Best validation accuracy: {best_val_acc:.4f}')
    print('Best combination of hyper-parameters:')
    display(model_params)


#### Model evaluation

Test data

In [ ]:
if which_model=='lr':
    model = deepcopy(lr_model[str(params)])

    test_accuracy = accuracy_score(
        y_true=labels_test,
        y_pred=model.predict(X=flatten_images_test)
    )
    print(f'Test accuracy: {test_accuracy:.4f}.')

<a id='lgb_model'></a>

### LightGBM

#### Model configuration

In [ ]:
if which_model=='lgb':
    # Creating grid of hyper-parameters for optimization:
    # lgb_grid = create_grid(params=LGB_PARAMS, random_search=True, n_samples=5)
    lgb_grid = create_grid(params=LGB_PARAMS, random_search=True, n_samples=1)

#### Model training

In [ ]:
if which_model=='lgb':
    lgb_model = {}
    best_val_acc, model_params = 0, None

    start_time = datetime.now()

    # Defining dataset for LightGBM estimation:
    train_data = lgb.Dataset(
        data=flatten_images_train.values, label=[l-1 for l in labels_train],
        params={'verbose': -1}
    )

    # Loop over combinations of hyper-parameters:
    for params in lgb_grid:
        params['num_class'] = len(label_dict)

        # Creating and fitting the model:
        lgb_model[str(params)] = lgb.train(
            params=params, train_set=train_data
        )

        # Predictions on validation set:
        val_preds = [np.argmax(p) for p in lgb_model[str(params)].predict(data=flatten_images_val.values)]

        # Model performance on validation set:
        new_acc = accuracy_score(y_true=[l-1 for l in labels_val], y_pred=val_preds)
        if new_acc >= best_val_acc:
            model_params = deepcopy(params)
            best_val_acc = new_acc
    
    end_time = datetime.now()
    elapsed_time = running_time(start_time=start_time, end_time=end_time)

    print(f'Best validation accuracy: {best_val_acc:.4f}')
    print('Best combination of hyper-parameters:')
    display(model_params)

#### Model evaluation

Test data

In [ ]:
if which_model=='lgb':
    model = deepcopy(lgb_model[str(model_params)])

    test_accuracy = accuracy_score(
        y_true=[l-1 for l in labels_test],
        y_pred=[np.argmax(p) for p in model.predict(data=flatten_images_test.values)]
    )
    print(f'Test accuracy: {test_accuracy:.4f}.')

<a id='ann'></a>

## Artificial neural networks

<a id='ann_config'></a>

### Neural network configuration

In [ ]:
if which_model=='ann':
    # Hidden layers hyper-parameters:
    neurons = [3000, 1500, 750]
    num_hidden = len(neurons)
    weights_initializer = ['glorot_uniform' for i in range(num_hidden)]
    bias_initializer = ['zeros' for i in range(num_hidden)]
    activation = ['relu' for i in range(num_hidden)]

    # Regularization hyper-parameters:
    input_dropout = 0.33
    dropout = [0.5 for i in range(num_hidden)]
    regul = ['l1' for i in range(num_hidden)]
    regul_param = [0.01 for i in range(num_hidden)]

    # Output layer hyper-parameters:
    output_activation = 'softmax'
    loss_function = 'categorical_crossentropy'

    # Fitting hyper-parameters:
    num_epochs = 100
    batch_size = 32
    opt = 'adam'
    opt_params = None
    patience = 30

    if opt == 'sgd':              
        opt = SGD(
            learning_rate=opt_params['learning_rate'], momentum=opt_params['momentum'],
            decay=opt_params['decay']
        )

    elif opt == 'adam_custom':
        opt = Adam(
            learning_rate=opt_params['learning_rate'], beta_1=opt_params['beta_1'],
            beta_2=opt_params['beta_2'], epsilon=opt_params['epsilon']
        )

    # Accuracy monitoring and early stopping:
    accuracy_callback = AccuracyEpochs(
        val_inputs=flatten_images_val,
        val_output=[img-1 for img in labels_val],
        early_stopping=True,
        patience=patience
    )

<a id='ann_construction'></a>

### Model construction

In [ ]:
if which_model=='ann':
    # Declaring model object:
    model = Sequential()

    # Dropout for the input layer:
    model.add(Dropout(input_dropout, input_shape=(flatten_images_train.shape[1],)))

    # Declaring first hidden layer:
    model.add(
        Dense(
            units=neurons[0],
            kernel_initializer=weights_initializer[0],
            bias_initializer=bias_initializer[0],
            kernel_regularizer=l1(l1=regul_param[0]) if regul[0]=='l1' else (l2(l2=regul_param[0]) if regul[0]=='l2' else None),
            activation=activation[0]
        )
    )
    model.add(Dropout(rate=dropout[0]))

    # Declaring additional hidden layers:
    for i in range(1, num_hidden):
        model.add(
            Dense(
                units=neurons[i],
                kernel_initializer=weights_initializer[i],
                bias_initializer=bias_initializer[i],
                kernel_regularizer=l1(l1=regul_param[i]) if regul[i]=='l1' else (l2(l2=regul_param[i]) if regul[i]=='l2' else None),
                activation=activation[i]
            )
        )
        model.add(Dropout(rate=dropout[i]))

    # Creating output layer:
    model.add(
        Dense(
            units=len(label_dict), activation=output_activation
        )
    )

    # Compiling the model:
    model.compile(
        loss=loss_function,
        optimizer=opt
    )

    # Model architecture:
    model.summary()

<a id='ann_training'></a>

### Model training

In [ ]:
if which_model=='ann':
    start_time = datetime.now()

    # Training the model:
    model = model.fit(
        x=flatten_images_train, 
        y=np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_train]),
        validation_data=(
            flatten_images_val,
            np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_val])
        ),
        epochs=num_epochs,
        batch_size=batch_size,
        shuffle=True,
        callbacks=[
            accuracy_callback
        ],
        verbose='auto'
    )

    end_time = datetime.now()
    elapsed_time = running_time(start_time=start_time, end_time=end_time)

<a id='ann_eval'></a>

### Model evaluation

#### Validation data

In [ ]:
if which_model=='ann':
    # Dataframe with metrics at each epoch:
    metrics_epoch = pd.DataFrame(model.history)
    metrics_epoch['epoch'] = [i+1 for i in range(len(metrics_epoch))]

    # Validation accuracy at each epoch:
    metrics_epoch['val_accuracy'] = accuracy_callback.val_accuracy

    print(f'Number of epochs used: {len(metrics_epoch)} (out of {num_epochs}).')
    best_val_acc = metrics_epoch.val_accuracy.max()
    print(f'Maximum validation accuracy: {best_val_acc:.4f}.')
    display(metrics_epoch.tail(10))

#### Test data

In [ ]:
if which_model=='ann':
    test_accuracy = accuracy_score(
        y_true=[l-1 for l in labels_test],
        y_pred=[np.argmax(p) for p in model.model.predict(flatten_images_test)]
    )
    print(f'Test accuracy: {test_accuracy:.4f}.')

In [ ]:
if which_model=='ann':
    model_params = {
        'hidden_layers': {
            'num_hidden': num_hidden,
            'neurons': neurons,
            'weights_initializer': weights_initializer,
            'bias_initializer': bias_initializer,
            'activation': activation
        },
        'regularization': {
            'input_dropout': input_dropout,
            'dropout': dropout,
            'regul': regul,
            'regul_param': regul_param
        },
        'ouput_layer': {
            'output_activation': output_activation,
            'loss_function': loss_function
        },
        'fitting': {
            'num_epochs': num_epochs,
            'batch_size': batch_size,
            'opt': opt,
            'opt_params': opt_params,
            'patience': patience
        }
    }

<a id='cnn'></a>

## Convolutional neural networks

In [ ]:
if (grey_scale) & (which_model in ['cnn', 'transfer']):
    images_train_scaled = np.array([img.reshape(224, 224, 1) for img in images_train_scaled])
    images_val_scaled = np.array([img.reshape(224, 224, 1) for img in images_val_scaled])
    images_test_scaled = np.array([img.reshape(224, 224, 1) for img in images_test_scaled])

<a id='cnn_config'></a>

### Neural network configuration

In [ ]:
if which_model=='cnn':
    # Convolutional layers hyper-parameters:
    input_shape = images_train_scaled[0].shape
    filters = [16]
    num_convs = len(filters)
    kernel_size = [(3, 3) for i in range(num_convs)]
    conv_padding = ['same' for i in range(num_convs)]
    conv_strides = [(2, 2) for i in range(num_convs)]
    conv_activation = ['relu' for i in range(num_convs)]
    conv_weights_init = ['glorot_uniform' for i in range(num_convs)]
    conv_bias_init = ['zeros' for i in range(num_convs)]
    conv_regul = [None for i in range(num_convs)]

    # Pooling layers hyper-parameters:
    pool_size = [(5, 5) for i in range(num_convs)]
    pool_padding = ['same' for i in range(num_convs)]
    pool_strides = [None for i in range(num_convs)]

    # Dropout of convolutional-pooling layers pairs:
    conv_dropout = [0.75 for i in range(num_convs)]

    # Hyper-parameters of fully-connected layers:
    global_pooling = False # Uses GlobalAveragePooling2D if True and Flatten otherwise.
    neurons = [512]
    num_hidden = len(neurons)
    dense_weights_init = ['glorot_uniform' for i in range(num_hidden)]
    dense_bias_init = ['zeros' for i in range(num_hidden)]
    dense_activation = ['relu' for i in range(num_hidden)]
    dense_dropout = [None for i in range(num_hidden)]
    dense_regul = [None for i in range(num_hidden)]
    dense_regul_param = [None for i in range(num_hidden)]

    # Output layer hyper-parameters:
    output_activation = 'softmax'
    loss_function = 'categorical_crossentropy'

    # Fitting hyper-parameters:
    num_epochs = 150
    batch_size = 64
    opt = 'adam'
    opt_params = None
    patience = 40

    if opt == 'sgd':              
        opt = SGD(
            learning_rate=opt_params['learning_rate'], momentum=opt_params['momentum'],
            decay=opt_params['decay']
        )

    elif opt == 'adam_custom':
        opt = Adam(
            learning_rate=opt_params['learning_rate'], beta_1=opt_params['beta_1'],
            beta_2=opt_params['beta_2'], epsilon=opt_params['epsilon']
        )

    # Accuracy monitoring and early stopping:
    accuracy_callback = AccuracyEpochs(
        val_inputs=images_val_scaled,
        val_output=[img-1 for img in labels_val],
        early_stopping=True,
        patience=patience
    )

<a id='cnn_construction'></a>

### Model construction

In [ ]:
if which_model=='cnn':
    # Declaring model object:
    model = Sequential()

    # Creating the first convolutional layer followed by max-pooling layer:
    model.add(
        Conv2D(
            input_shape=input_shape,
            filters=filters[0],
            kernel_size=kernel_size[0],
            padding=conv_padding[0],
            strides=conv_strides[0],
            activation=conv_activation[0],
            kernel_initializer=conv_weights_init[0],
            bias_initializer=conv_bias_init[0],
            kernel_regularizer=conv_regul[0]
        )
    )
    model.add(MaxPool2D(pool_size=pool_size[0], padding=pool_padding[0], strides=pool_strides[0]))
    if conv_dropout[0] is not None:
        model.add(Dropout(rate=conv_dropout[0]))
    
    # Declaring additional pairs of convolutional and max-pooling layers:
    for i in range(1, num_convs):
        model.add(
            Conv2D(
              filters=filters[i],
              kernel_size=kernel_size[i],
              padding=conv_padding[i],
              strides=conv_strides[i],
              activation=conv_activation[i],
              kernel_initializer=conv_weights_init[i],
              bias_initializer=conv_bias_init[i],
              kernel_regularizer=conv_regul[i]
            )
        )
        model.add(MaxPool2D(pool_size=pool_size[i], padding=pool_padding[i], strides=pool_strides[i]))
        if conv_dropout[i] is not None:
            model.add(Dropout(rate=conv_dropout[i]))

    # Flatten hidden layers previous to output layer:
    model.add(
        GlobalAveragePooling2D() if global_pooling else Flatten()
    )

    # Creating fully-connected layers:
    for i in range(0, num_hidden):
        model.add(
            Dense(
                units=neurons[i],
                kernel_initializer=dense_weights_init[i],
                bias_initializer=dense_bias_init[i],
                kernel_regularizer=l1(l1=dense_regul_param[i]) if dense_regul[i]=='l1' else (l2(l2=dense_regul_param[i]) if dense_regul[i]=='l2' else None),
                activation=dense_activation[i]
            )
        )
        if dense_dropout[i] is not None:
            model.add(Dropout(rate=dense_dropout[i]))

    # Creating output layer:
    model.add(
        Dense(
            units=len(label_dict), activation=output_activation
        )
    )

    # Compiling the model:
    model.compile(
        loss=loss_function,
        optimizer=opt
    )

    # Model architecture:
    model.summary()

<a id='cnn_training'></a>

### Model training

In [ ]:
if which_model=='cnn':
    start_time = datetime.now()

    # Training the model:
    model = model.fit(
        x=images_train_scaled, 
        y=np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_train]),
        validation_data=(
            images_val_scaled,
            np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_val])
        ),
        epochs=num_epochs,
        batch_size=batch_size,
        shuffle=True,
        callbacks=[
            accuracy_callback
        ],
        verbose='auto'
    )

    end_time = datetime.now()
    elapsed_time = running_time(start_time=start_time, end_time=end_time)

<a id='cnn_eval'></a>

### Model evaluation

#### Validation data

In [ ]:
if which_model=='cnn':
    # Dataframe with metrics at each epoch:
    metrics_epoch = pd.DataFrame(model.history)
    metrics_epoch['epoch'] = [i+1 for i in range(len(metrics_epoch))]

    # Validation accuracy at each epoch:
    metrics_epoch['val_accuracy'] = accuracy_callback.val_accuracy

    print(f'Number of epochs used: {len(metrics_epoch)} (out of {num_epochs}).')
    best_val_acc = metrics_epoch.val_accuracy.max()
    print(f'Maximum validation accuracy: {best_val_acc:.4f}.')
    display(metrics_epoch.tail(10))

#### Test data

In [ ]:
if which_model=='cnn':
    test_accuracy = accuracy_score(
        y_true=[l-1 for l in labels_test],
        y_pred=[np.argmax(p) for p in model.model.predict(images_test_scaled)]
    )
    print(f'Test accuracy: {test_accuracy:.4f}.')

In [ ]:
if which_model=='cnn':
		model_params = {
				'conv_layers': {
						'input_shape': input_shape,
						'filters': filters,
						'num_convs': num_convs,
						'kernel_size': kernel_size,
						'conv_padding': conv_padding,
						'conv_strides': conv_strides,
						'conv_activation': conv_activation,
						'conv_weights_init': conv_weights_init,
						'conv_bias_init': conv_bias_init,
						'conv_regul': str(conv_regul),
						'conv_dropout': conv_dropout
				},
				'pool_layers': {
						'pool_size': pool_size,
						'pool_padding': pool_padding,
						'pool_strides': pool_strides
				},
				'fully_layers': {
						'global_pooling': global_pooling,
						'neurons': neurons,
						'num_hidden': num_hidden,
						'dense_weights_init': dense_weights_init,
						'dense_bias_init': dense_bias_init,
						'dense_activation': dense_activation,
						'dense_dropout': dense_dropout,
						'dense_regul': dense_regul,
						'dense_regul_param': dense_regul_param
				},
				'output_layer': {
						'output_activation': output_activation,
						'loss_function': loss_function
				},
				'fitting': {
						'num_epochs': num_epochs,
						'batch_size': batch_size,
						'opt': opt,
						'opt_params': opt_params,
						'patience': patience
				}
		}

<a id='transfer_learning'></a>

## Transfer learning

<a id='transfer_config'></a>

### Neural network configuration

In [ ]:
if which_model=='transfer':
    # Base model:
    which_base_model = 'xception' # ['resnet', 'inception', 'vgg16', 'vgg19', 'xception']

    # Base model configurations:
    base_model_conf = {
        'include_top': False,
        'weights': 'imagenet', # [None, 'imagenet']
        'input_shape': images_train_scaled[0].shape,
        'pooling': 'avg', # [None, 'avg', 'max']
        'classes': len(label_dict)
    }

    # Declaring the pre-trained model:
    base_models_dict = {
        'resnet': ResNet50(**base_model_conf),
        'inception': InceptionV3(**base_model_conf),
        'vgg16': VGG16(**base_model_conf),
        'vgg19': VGG19(**base_model_conf),
        'xception': Xception(**base_model_conf)
    }
    base_model = base_models_dict[which_base_model]

    # Freezing parameters of pre-trained model:
    base_model.trainable = False

    # Hyper-parameters of fully-connected layers:
    global_pooling = False # Uses GlobalAveragePooling2D if True and Flatten otherwise.
    neurons = [1024, 512]
    num_hidden = len(neurons)
    dense_weights_init = ['glorot_uniform' for i in range(num_hidden)]
    dense_bias_init = ['zeros' for i in range(num_hidden)]
    dense_activation = ['relu' for i in range(num_hidden)]
    dense_dropout = [None for i in range(num_hidden)]
    dense_regul = [None for i in range(num_hidden)]
    dense_regul_param = [None for i in range(num_hidden)]

    # Output layer hyper-parameters:
    output_activation = 'softmax'
    loss_function = 'categorical_crossentropy'

    # Fitting hyper-parameters:
    num_epochs = 50
    batch_size = 64
    opt = 'adam'
    opt_params = None
    patience = 12

    if opt == 'sgd':              
        opt = SGD(
            learning_rate=opt_params['learning_rate'], momentum=opt_params['momentum'],
            decay=opt_params['decay']
        )

    elif opt == 'adam_custom':
        opt = Adam(
            learning_rate=opt_params['learning_rate'], beta_1=opt_params['beta_1'],
            beta_2=opt_params['beta_2'], epsilon=opt_params['epsilon']
        )

    # Accuracy monitoring and early stopping:
    accuracy_callback = AccuracyEpochs(
        val_inputs=images_val_scaled,
        val_output=[img-1 for img in labels_val],
        early_stopping=True,
        patience=patience
    )

<a id='transfer_construction'></a>

### Model construction

In [ ]:
if which_model=='transfer':
    K.clear_session()

    # Declaring model object:
    model = Sequential()

    # Adding layers from pre-trained model:
    model.add(
        base_model
    )

    # Flatten hidden layers previous to output layer:
    model.add(
        GlobalAveragePooling2D() if global_pooling else Flatten()
    )

    # Creating fully-connected layers:
    for i in range(0, num_hidden):
        model.add(
            Dense(
                units=neurons[i],
                kernel_initializer=dense_weights_init[i],
                bias_initializer=dense_bias_init[i],
                kernel_regularizer=l1(l1=dense_regul_param[i]) if dense_regul[i]=='l1' else (l2(l2=dense_regul_param[i]) if dense_regul[i]=='l2' else None),
                activation=dense_activation[i]
            )
        )
        if dense_dropout[i] is not None:
            model.add(Dropout(rate=dense_dropout[i]))

    # Creating output layer:
    model.add(
        Dense(
            units=len(label_dict), activation=output_activation
        )
    )

    # Compiling the model:
    model.compile(
        loss=loss_function,
        optimizer=opt
    )

    # Model architecture:
    model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 23,487,534
Trainable params: 2,626,054
Non-trainable params: 20,861,480
_________________________________________________________________


<a id='transfer_training'></a>

### Model training

In [ ]:
if which_model=='transfer':
    start_time = datetime.now()

    # Training the model:
    model = model.fit(
        x=images_train_scaled, 
        y=np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_train]),
        validation_data=(
            images_val_scaled,
            np.array([to_categorical(img-1, num_classes=len(label_dict)) for img in labels_val])
        ),
        epochs=num_epochs,
        batch_size=batch_size,
        shuffle=True,
        callbacks=[
            accuracy_callback
        ],
        verbose='auto'
    )

    end_time = datetime.now()
    elapsed_time = running_time(start_time=start_time, end_time=end_time)

Epoch 1/50
42/42 [==============================] - 28s 445ms/step - loss: 0.8518 - val_loss: 0.4792
Epoch 2/50
42/42 [==============================] - 16s 379ms/step - loss: 0.4226 - val_loss: 0.4036
Epoch 3/50
42/42 [==============================] - 16s 386ms/step - loss: 0.2491 - val_loss: 0.3535
Epoch 4/50
42/42 [==============================] - 15s 355ms/step - loss: 0.1540 - val_loss: 0.4597
Epoch 5/50
42/42 [==============================] - 15s 352ms/step - loss: 0.1109 - val_loss: 0.5347
Epoch 6/50
42/42 [==============================] - 15s 355ms/step - loss: 0.0596 - val_loss: 0.4026
Epoch 7/50
42/42 [==============================] - 16s 375ms/step - loss: 0.0264 - val_loss: 0.5209
Epoch 8/50
42/42 [==============================] - 15s 349ms/step - loss: 0.0380 - val_loss: 0.5551
Epoch 9/50
42/42 [==============================] - 15s 351ms/step - loss: 0.0277 - val_loss: 0.5626
Epoch 10/50
42/42 [==============================] - 15s 351ms/step - loss: 0.0225 - val_lo

<a id='transfer_eval'></a>

### Model evaluation

#### Validation data

In [ ]:
if which_model=='transfer':
    # Dataframe with metrics at each epoch:
    metrics_epoch = pd.DataFrame(model.history)
    metrics_epoch['epoch'] = [i+1 for i in range(len(metrics_epoch))]

    # Validation accuracy at each epoch:
    metrics_epoch['val_accuracy'] = accuracy_callback.val_accuracy

    print(f'Number of epochs used: {len(metrics_epoch)} (out of {num_epochs}).')
    best_val_acc = metrics_epoch.val_accuracy.max()
    print(f'Maximum validation accuracy: {best_val_acc:.4f}.')
    display(metrics_epoch.tail(10))

Number of epochs used: 23 (out of 50).
Maximum validation accuracy: 0.8811.


,loss,val_loss,epoch,val_accuracy
13,0.003571,0.543864,14,0.868902
14,0.002425,0.541905,15,0.856707
15,0.002300,0.555146,16,0.862805
16,0.002151,0.567519,17,0.853659
17,0.002457,0.551676,18,0.862805
18,0.001252,0.553385,19,0.871951
19,0.001350,0.555644,20,0.868902
20,0.000999,0.558513,21,0.865854
21,0.000814,0.562058,22,0.871951
22,0.000814,0.570431,23,0.868902


#### Test data

In [ ]:
if which_model=='transfer':
    test_accuracy = accuracy_score(
        y_true=[l-1 for l in labels_test],
        y_pred=[np.argmax(p) for p in model.model.predict(images_test_scaled)]
    )
    print(f'Test accuracy: {test_accuracy:.4f}.')

Test accuracy: 0.8469.


In [ ]:
if which_model=='transfer':
		model_params = {
				'base_model': {
						'which_base_model': which_base_model,
						'base_model_conf': base_model_conf
				},
				'fully_layers': {
						'global_pooling': global_pooling,
						'neurons': neurons,
						'num_hidden': num_hidden,
						'dense_weights_init': dense_weights_init,
						'dense_bias_init': dense_bias_init,
						'dense_activation': dense_activation,
						'dense_dropout': dense_dropout,
						'dense_regul': dense_regul,
						'dense_regul_param': dense_regul_param
				},
				'output_layer': {
						'output_activation': output_activation,
						'loss_function': loss_function
				},
				'fitting': {
					'num_epochs': num_epochs,
					'batch_size': batch_size,
					'opt': opt,
					'opt_params': opt_params,
					'patience': patience
				}
		}

In [ ]:
if which_model=='transfer':
		model_params = {
				'base_model': {
						'which_base_model': which_base_model,
						'base_model_conf': base_model_conf
				},
				'fully_layers': {
						'global_pooling': global_pooling,
						'neurons': neurons,
						'num_hidden': num_hidden,
						'dense_weights_init': dense_weights_init,
						'dense_bias_init': dense_bias_init,
						'dense_activation': dense_activation,
						'dense_dropout': dense_dropout,
						'dense_regul': dense_regul,
						'dense_regul_param': dense_regul_param
				},
				'output_layer': {
						'output_activation': output_activation,
						'loss_function': loss_function
				},
				'fitting': {
					'num_epochs': num_epochs,
					'batch_size': batch_size,
					'opt': opt,
					'opt_params': opt_params,
					'patience': patience
				}
		}

<a id='exports'></a>

## Exporting outcomes

In [ ]:
if which_model=='lgb':
    model_params['max_depth'] = int(model_params['max_depth'])
    model_params['num_class'] = int(model_params['num_class'])
    model_params['num_iterations'] = int(model_params['num_iterations'])

model_assess = {
		"data_prep": {
				"grey_scale": grey_scale,
				"keras_augment": keras_augment,
				"custom_augment": custom_augment,
				"scale": scale,
				'augment_params': AUGMENT_PARAMS
		},
		"data_modeling": {
				"which_model": which_model,
				"model_params": model_params
		},
		"model_eval": {

				"best_val_accuracy": best_val_acc,
				"test_accuracy": test_accuracy,
				"running_time": elapsed_time
		}
}

if export:
    with open(f'../experiments/model_assess_{experiment_id}.json', 'w') as json_file:
        json.dump(model_assess, json_file, indent=2)
    
if (export) & (export_artifacts):
		# if which_model in ['ann', 'cnn', 'transfer']:
		# 		model.model.save(f'../artifacts/model_{experiment_id}')
		# else:
		# 		pickle.dump(model, open(f'../artifacts/model_{experiment_id}.pickle', 'wb'))
		pickle.dump(model, open(f'../artifacts/model_{experiment_id}.pickle', 'wb'))

if (export) & (which_model in ['ann', 'cnn', 'transfer']):
		metrics_epoch.to_csv(f'../experiments/metrics_epoch_{experiment_id}.csv', index=False)

<a id='model_assess'></a>

## Model assessment

<a id='model_perf'></a>

### Model performance

In [ ]:
model_assess = {}

# Loop over experiments metadata:
for f in [f for f in os.listdir('../experiments/') if 'model_assess' in f]:
    exp_id = f.split('model_assess_')[1].split('.')[0]
    with open(f'../experiments/{f}', 'rb') as json_file:
        model_assess[exp_id] = json.load(json_file)

In [ ]:
# Dataframe with metadata and model performance:
model_assess_df = pd.DataFrame(data={
    'experiment_id': [e for e in model_assess],
    'which_model': [model_assess[e]['data_modeling']['which_model'] for e in model_assess],
    'test_accuracy': [model_assess[e]['model_eval']['test_accuracy'] for e in model_assess],
    'val_accuracy': [model_assess[e]['model_eval']['best_val_accuracy'] for e in model_assess],
    'running_time': [model_assess[e]['model_eval']['running_time'] for e in model_assess],
    'grey_scale': [model_assess[e]['data_prep']['grey_scale'] for e in model_assess],
    'scale': [model_assess[e]['data_prep']['scale'] for e in model_assess],
    'augment_params': [model_assess[e]['data_prep']['augment_params'] for e in model_assess],
    'custom_augment': [model_assess[e]['data_prep']['custom_augment'] for e in model_assess],
    'keras_augment': [model_assess[e]['data_prep']['keras_augment'] for e in model_assess],
})

print(f'Shape of model_assess: {model_assess_df.shape}.')
model_assess_df.tail(25)

Shape of model_assess: (120, 10).


,experiment_id,which_model,test_accuracy,val_accuracy,running_time,grey_scale,scale,augment_params,custom_augment,keras_augment
95,1665109289,cnn,0.712297,0.728659,874.165079,False,True,{},False,False
96,1666835433,cnn,0.698376,0.734756,1040.294468,False,True,{},False,False
97,1667266671,cnn,0.712297,0.704268,957.779397,False,True,{},False,False
98,1666834302,cnn,0.540603,0.527439,905.237946,False,True,{},False,False
99,1667269579,cnn,0.670534,0.667683,830.159196,False,True,{},False,False
100,1667267880,cnn,0.663573,0.689024,945.506683,False,True,{},False,False
101,1667849187,transfer,0.487239,0.484756,12681.881981,False,True,{},False,False
102,1667925793,transfer,0.819026,0.823171,5730.579648,False,True,{},False,False
103,1668086604,transfer,0.758701,0.743902,11518.109721,False,True,{},False,False
104,1668086601,transfer,0.740139,0.771341,11512.133910,False,True,{},False,False


#### Best model

In [ ]:
# Identification of best model so far:
best_model = model_assess_df[model_assess_df['test_accuracy']==max(model_assess_df['test_accuracy'])]['experiment_id'].iloc[0]
print(f'Best model: {best_model}')
model_assess[best_model]

Best model: 1670952710


{'data_prep': {'grey_scale': False,
  'keras_augment': True,
  'custom_augment': False,
  'scale': True,
  'augment_params': {'augmentation_factor': 1.5,
   'oper_params': {'brightness_range': [0.5, 1.5],
    'horizontal_flip': True,
    'vertical_flip': True,
    'rotation_range': 180,
    'zoom_range': [0.5, 1.75],
    'fill_mode': 'nearest'}}},
 'data_modeling': {'which_model': 'transfer',
  'model_params': {'base_model': {'which_base_model': 'xception',
    'base_model_conf': {'include_top': False,
     'weights': 'imagenet',
     'input_shape': [224, 224, 3],
     'pooling': 'avg',
     'classes': 6}},
   'fully_layers': {'global_pooling': False,
    'neurons': [1024, 512],
    'num_hidden': 2,
    'dense_weights_init': ['glorot_uniform', 'glorot_uniform'],
    'dense_bias_init': ['zeros', 'zeros'],
    'dense_activation': ['relu', 'relu'],
    'dense_dropout': [None, None],
    'dense_regul': [None, None],
    'dense_regul_param': [None, None]},
   'output_layer': {'output_activa

<a id='nn_arch'></a>

### Neural network architectures

In [ ]:
pd.set_option('display.max_columns', None)

#### Artificial neural networks

In [ ]:
anns = [e for e in model_assess if model_assess[e]['data_modeling']['which_model']=='ann']

# Artificial neural networks architectures:
ann_arch = pd.DataFrame(data={
    'experiment_id': anns,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in anns],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in anns],
    'val_accuracy': [model_assess[exp_id]['model_eval']['best_val_accuracy'] for exp_id in anns],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in anns],
    'neurons': [model_assess[exp_id]['data_modeling']['model_params']['hidden_layers']['neurons'] for exp_id in anns],
    'input_dropout': [model_assess[exp_id]['data_modeling']['model_params']['regularization']['input_dropout'] for exp_id in anns],
    'dropout': [model_assess[exp_id]['data_modeling']['model_params']['regularization']['dropout'][0] for exp_id in anns],
    'regul': [model_assess[exp_id]['data_modeling']['model_params']['regularization']['regul'][0] for exp_id in anns],
    'regul_param': [model_assess[exp_id]['data_modeling']['model_params']['regularization']['regul_param'][0] for exp_id in anns],
    'batch_size': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['batch_size'] for exp_id in anns],
    'num_epochs': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['num_epochs'] for exp_id in anns],
    'patience': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['patience'] for exp_id in anns]
})

print(f'Shape of ann_arch: {ann_arch.shape}.')
ann_arch

Shape of ann_arch: (4, 13).


,experiment_id,grey_scale,test_accuracy,val_accuracy,running_time,neurons,input_dropout,dropout,regul,regul_param,batch_size,num_epochs,patience
0,1658619569,True,0.250580,0.253049,5428.582358,"[5000, 2500]",0.33,0.5,l1,0.01,32,100,30
1,1658614393,True,0.250580,0.253049,3471.462878,"[5000, 2500]",0.33,0.5,l1,0.01,32,100,20
2,1658628829,True,0.250580,0.253049,2968.295533,"[3000, 1500, 750]",0.33,0.5,l1,0.01,32,100,30
3,1658601048,True,0.171694,0.253049,5790.226852,[5000],0.33,0.5,l1,0.01,32,100,20


#### Convolutional neural networks

In [ ]:
cnns = [e for e in model_assess if model_assess[e]['data_modeling']['which_model']=='cnn']

# Artificial neural networks architectures:
cnn_arch = pd.DataFrame(data={
    'experiment_id': cnns,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in cnns],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in cnns],
    'val_accuracy': [model_assess[exp_id]['model_eval']['best_val_accuracy'] for exp_id in cnns],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in cnns],
    'filters': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['filters'] for exp_id in cnns],
    'kernel_size': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['kernel_size']] for exp_id in cnns],
    'conv_padding': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_padding'] for exp_id in cnns],
    'conv_strides': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_strides']] for exp_id in cnns],
    'pool_size': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_size']] for exp_id in cnns],
    'pool_padding': [model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_padding'] for exp_id in cnns],
    'pool_strides': [model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_strides'] for exp_id in cnns],
    'conv_dropout': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_dropout'] for exp_id in cnns],
    'conv_regul': [str(model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_regul']) for exp_id in cnns],
    'global_pooling': [model_assess[exp_id]['data_modeling']['model_params']['fully_layers'].get('global_pooling', False) for exp_id in cnns],
    'neurons': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['neurons'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_dropout': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_dropout'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_regul': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_regul_param': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul_param'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'batch_size': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['batch_size'] for exp_id in cnns],
    'num_epochs': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['num_epochs'] for exp_id in cnns],
    'patience': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['patience'] for exp_id in cnns]
})

print(f'Shape of cnn_ach: {cnn_arch.shape}.')
cnn_arch.tail(30)

Shape of cnn_ach: (90, 22).


,experiment_id,grey_scale,test_accuracy,val_accuracy,running_time,filters,kernel_size,conv_padding,conv_strides,pool_size,pool_padding,pool_strides,conv_dropout,conv_regul,global_pooling,neurons,dense_dropout,dense_regul,dense_regul_param,batch_size,num_epochs,patience
60,1664049037,False,0.619490,0.603659,137.274779,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,25,5
61,1664049251,False,0.693735,0.676829,215.270973,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,45,10
62,1664049569,False,0.707657,0.679878,442.547935,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,60,60
63,1664052530,False,0.698376,0.695122,471.557851,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,200,30
64,1664051600,False,0.716937,0.713415,622.399275,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,150,40
65,1664050080,False,0.682135,0.685976,322.266824,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,65,20
66,1664416719,False,0.707657,0.713415,1078.455357,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,150,40
67,1664418505,False,0.716937,0.716463,979.144561,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,150,40
68,1664500584,False,0.698376,0.710366,600.148675,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[0.2],[None],[None],64,150,40
69,1664502175,False,0.716937,0.713415,680.650610,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[0.5],[None],[None],64,150,40


In [ ]:
### https://docs.google.com/spreadsheets/d/1i1f_Y59TmBjFEd782Rkhid_90g_0EjC9/edit#gid=879866560

In [ ]:
drop_expers = [
    '1664041857', '1664042408', '1664045001', '1664045551', '1664046353', '1664416719',
    '1664051600', '1664502175', '1664743067', '1665103709', '1665104975', '1664418505',
    '1665109289'
]
cnn_arch[~(cnn_arch['experiment_id'].isin(drop_expers))].sort_values('test_accuracy').tail(5)

,experiment_id,grey_scale,test_accuracy,val_accuracy,running_time,filters,kernel_size,conv_padding,conv_strides,pool_size,pool_padding,pool_strides,conv_dropout,conv_regul,global_pooling,neurons,dense_dropout,dense_regul,dense_regul_param,batch_size,num_epochs,patience
50,1664038643,False,0.712297,0.707317,386.030503,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],128,90,20
86,1667266671,False,0.712297,0.704268,957.779397,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,150,40
79,1665101996,False,0.714617,0.731707,970.429811,[16],[3],[same],[2],[5],[same],[None],[0.75],['l1'],False,[512],[0.5],[None],[None],64,150,40
70,1664503115,False,0.716937,0.716463,653.316867,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[0.5],[None],[None],64,150,40
82,1665107942,False,0.716937,0.756098,987.628679,[16],[3],[same],[2],[5],[same],[None],[0.75],[None],False,[512],[None],[None],[None],64,150,40


Convolutional layers

In [ ]:
# Artificial neural networks architectures:
cnn_convs = pd.DataFrame(data={
    'experiment_id': cnns,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in cnns],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in cnns],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in cnns],
    'conv_dropout': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_dropout'] for exp_id in cnns],
    'filters': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['filters'] for exp_id in cnns],
    'kernel_size': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['kernel_size']] for exp_id in cnns],
    'conv_padding': [model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_padding'] for exp_id in cnns],
    'conv_strides': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['conv_layers']['conv_strides']] for exp_id in cnns],
    'pool_size': [[l[0] for l in model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_size']] for exp_id in cnns],
    'pool_padding': [model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_padding'] for exp_id in cnns],
    'pool_strides': [model_assess[exp_id]['data_modeling']['model_params']['pool_layers']['pool_strides'] for exp_id in cnns]
})

print(f'Shape of cnn_convs: {cnn_convs.shape}.')
cnn_convs

Shape of cnn_convs: (90, 12).


,experiment_id,grey_scale,test_accuracy,running_time,conv_dropout,filters,kernel_size,conv_padding,conv_strides,pool_size,pool_padding,pool_strides
0,1658963546,True,0.250580,1713.870751,[None],[32],[3],[valid],[1],[2],[valid],[None]
1,1659044562,True,0.389791,3292.819340,"[None, None]","[32, 32]","[3, 3]","[valid, valid]","[1, 1]","[2, 2]","[valid, valid]","[None, None]"
2,1659053572,True,0.320186,5676.767859,"[None, None]","[64, 32]","[3, 3]","[valid, valid]","[1, 1]","[2, 2]","[valid, valid]","[None, None]"
3,1659115013,True,0.250580,3763.123868,"[None, None]","[64, 32]","[3, 3]","[valid, valid]","[1, 1]","[2, 2]","[valid, valid]","[None, None]"
4,1659187852,False,0.533643,3237.584704,[None],[64],[3],[valid],[1],[2],[valid],[None]
...,...,...,...,...,...,...,...,...,...,...,...,...
85,1666834302,False,0.540603,905.237946,[0.75],[16],[3],[same],[2],[5],[same],[None]
86,1666835433,False,0.698376,1040.294468,[0.75],[16],[3],[same],[2],[5],[same],[None]
87,1667266671,False,0.712297,957.779397,[0.75],[16],[3],[same],[2],[5],[same],[None]
88,1667267880,False,0.663573,945.506683,[0.75],[16],[3],[same],[2],[5],[same],[None]


Fully-connected layers

In [ ]:
# Artificial neural networks architectures:
cnn_fully = pd.DataFrame(data={
    'experiment_id': cnns,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in cnns],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in cnns],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in cnns],
    'neurons': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['neurons'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_dropout': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_dropout'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_regul': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ],
    'dense_regul_param': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul_param'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in cnns
     ]
})

print(f'Shape of cnn_fully: {cnn_fully.shape}.')
cnn_fully

Shape of cnn_fully: (90, 8).


,experiment_id,grey_scale,test_accuracy,running_time,neurons,dense_dropout,dense_regul,dense_regul_param
0,1658963546,True,0.250580,1713.870751,[128],[0.2],[l1],[0.01]
1,1659044562,True,0.389791,3292.819340,[128],[0.2],[l1],[0.01]
2,1659053572,True,0.320186,5676.767859,[128],[0.2],[l1],[0.01]
3,1659115013,True,0.250580,3763.123868,"[128, 64]","[0.2, 0.2]","[l1, l1]","[0.01, 0.01]"
4,1659187852,False,0.533643,3237.584704,None,None,None,None
...,...,...,...,...,...,...,...,...
85,1666834302,False,0.540603,905.237946,[512],[None],[None],[None]
86,1666835433,False,0.698376,1040.294468,[512],[None],[None],[None]
87,1667266671,False,0.712297,957.779397,[512],[None],[None],[None]
88,1667267880,False,0.663573,945.506683,[512],[None],[None],[None]


Fitting parameters

In [ ]:
# Artificial neural networks architectures:
cnn_fitting = pd.DataFrame(data={
    'experiment_id': cnns,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in cnns],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in cnns],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in cnns],
    'batch_size': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['batch_size'] for exp_id in cnns],
    'num_epochs': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['num_epochs'] for exp_id in cnns],
    'patience': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['patience'] for exp_id in cnns]
})

print(f'Shape of cnn_fitting: {cnn_fitting.shape}.')
cnn_fitting

Shape of cnn_fitting: (90, 7).


,experiment_id,grey_scale,test_accuracy,running_time,batch_size,num_epochs,patience
0,1658963546,True,0.250580,1713.870751,64,30,20
1,1659044562,True,0.389791,3292.819340,64,45,20
2,1659053572,True,0.320186,5676.767859,64,45,20
3,1659115013,True,0.250580,3763.123868,64,45,20
4,1659187852,False,0.533643,3237.584704,64,45,20
...,...,...,...,...,...,...,...
85,1666834302,False,0.540603,905.237946,64,150,40
86,1666835433,False,0.698376,1040.294468,64,150,40
87,1667266671,False,0.712297,957.779397,64,150,40
88,1667267880,False,0.663573,945.506683,64,150,40


#### Transfer learning

In [ ]:
transf = [e for e in model_assess if model_assess[e]['data_modeling']['which_model']=='transfer']

# Artificial neural networks architectures:
transf_arch = pd.DataFrame(data={
    'experiment_id': transf,
    'grey_scale': [model_assess[exp_id]['data_prep']['grey_scale'] for exp_id in transf],
    'test_accuracy': [model_assess[exp_id]['model_eval']['test_accuracy'] for exp_id in transf],
    'val_accuracy': [model_assess[exp_id]['model_eval']['best_val_accuracy'] for exp_id in transf],
    'running_time': [model_assess[exp_id]['model_eval']['running_time'] for exp_id in transf],
    'which_base_model': [model_assess[exp_id]['data_modeling']['model_params']['base_model']['which_base_model'] for exp_id in transf],
    'weights': [model_assess[exp_id]['data_modeling']['model_params']['base_model']['base_model_conf']['weights'] for exp_id in transf],
    'pooling': [model_assess[exp_id]['data_modeling']['model_params']['base_model']['base_model_conf']['pooling'] for exp_id in transf],
    'global_pooling': [model_assess[exp_id]['data_modeling']['model_params']['fully_layers'].get('global_pooling', False) for exp_id in transf],
    'neurons': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['neurons'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in transf
     ],
    'dense_dropout': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_dropout'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in transf
     ],
    'dense_regul': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in transf
     ],
    'dense_regul_param': [
        model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['dense_regul_param'] if \
          model_assess[exp_id]['data_modeling']['model_params']['fully_layers']['num_hidden'] > 0 else None
        for exp_id in transf
     ],
    'batch_size': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['batch_size'] for exp_id in transf],
    'num_epochs': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['num_epochs'] for exp_id in transf],
    'patience': [model_assess[exp_id]['data_modeling']['model_params']['fitting']['patience'] for exp_id in transf]
})

print(f'Shape of transf_arch: {transf_arch.shape}.')
transf_arch.tail(30)

Shape of transf_arch: (23, 16).


,experiment_id,grey_scale,test_accuracy,val_accuracy,running_time,which_base_model,weights,pooling,global_pooling,neurons,dense_dropout,dense_regul,dense_regul_param,batch_size,num_epochs,patience
0,1667849187,False,0.487239,0.484756,12681.881981,resnet,imagenet,None,False,[512],[None],[None],[None],64,50,12
1,1667925793,False,0.819026,0.823171,5730.579648,inception,imagenet,None,False,[512],[None],[None],[None],64,50,12
2,1668086604,False,0.758701,0.743902,11518.109721,vgg19,imagenet,None,False,[512],[None],[None],[None],64,16,4
3,1668086601,False,0.740139,0.771341,11512.133910,vgg16,imagenet,None,False,[512],[None],[None],[None],64,16,4
4,1668098799,False,0.839907,0.902439,9850.818761,xception,imagenet,None,False,[512],[None],[None],[None],64,50,12
5,1668177518,False,0.842227,0.896341,10125.182795,xception,imagenet,avg,False,[512],[None],[None],[None],64,50,12
6,1668177482,False,0.837587,0.899390,10202.660344,xception,imagenet,max,False,[512],[None],[None],[None],64,50,12
7,1668275597,False,0.825986,0.905488,10995.099378,xception,imagenet,avg,False,[256],[None],[None],[None],64,50,12
8,1668301721,False,0.846868,0.896341,11394.099340,xception,imagenet,avg,False,[1024],[None],[None],[None],64,50,12
9,1668352819,False,0.844548,0.902439,12443.858193,xception,imagenet,avg,False,"[512, 256]","[None, None]","[None, None]","[None, None]",64,50,12


In [ ]:
drop_expers = []
transf_arch[~(transf_arch['experiment_id'].isin(drop_expers))].sort_values('test_accuracy').tail(5)

,experiment_id,grey_scale,test_accuracy,val_accuracy,running_time,which_base_model,weights,pooling,global_pooling,neurons,dense_dropout,dense_regul,dense_regul_param,batch_size,num_epochs,patience
17,1668697944,False,0.842227,0.896341,13279.605979,xception,imagenet,avg,False,"[1024, 512]","[None, None]","[None, None]","[None, None]",64,60,16
9,1668352819,False,0.844548,0.902439,12443.858193,xception,imagenet,avg,False,"[512, 256]","[None, None]","[None, None]","[None, None]",64,50,12
8,1668301721,False,0.846868,0.896341,11394.099340,xception,imagenet,avg,False,[1024],[None],[None],[None],64,50,12
10,1668353741,False,0.846868,0.902439,11722.537886,xception,imagenet,avg,False,"[1024, 512]","[None, None]","[None, None]","[None, None]",64,50,12
18,1670952710,False,0.849188,0.905488,397.124902,xception,imagenet,avg,False,"[1024, 512]","[None, None]","[None, None]","[None, None]",64,50,12
